<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Instrument-level-JWST-queries" data-toc-modified-id="Instrument-level-JWST-queries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Instrument-level JWST queries</a></span></li><li><span><a href="#Query-executed-data-from-ERS-TEMPLATES-program" data-toc-modified-id="Query-executed-data-from-ERS-TEMPLATES-program-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Query executed data from ERS-TEMPLATES program</a></span><ul class="toc-item"><li><span><a href="#Parse-fields" data-toc-modified-id="Parse-fields-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Parse fields</a></span></li><li><span><a href="#Split-into-our-own-&quot;associations&quot;-and-make-fancy-figures" data-toc-modified-id="Split-into-our-own-&quot;associations&quot;-and-make-fancy-figures-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Split into our own "associations" and make fancy figures</a></span></li></ul></li></ul></div>

# Instrument-level JWST queries

In [ ]:
import numpy as np
import astropy.units as u

import mastquery
from mastquery import overlaps, query, jwst
import astropy.time

import logging
logging.basicConfig(level=logging.INFO,
                    handlers=[logging.StreamHandler(),
                              logging.FileHandler('/tmp/mastquery.log')], 
                    format = "%(module)s.%(funcName)s : %(levelname)s : %(message)s")

print('mastquery version: ', mastquery.__version__, mastquery.__version_hash__)

# Query executed data from ERS-TEMPLATES program

ERS-1355


In [ ]:
filters = []

# filter on program ID
filters += jwst.make_program_filter([1355]) 

# cal and rate files
filters += jwst.make_query_filter('productLevel', values=['2a','2b'])
filters += jwst.make_query_filter('filename', text='%_[cr]a%[le].fits')

# Just guided exposures
filters += jwst.make_query_filter('pcs_mode', values=['FINEGUIDE'])

# filters += jwst.make_query_filter('expstart', range=[59744.5, 59764.5])

res = jwst.query_all_jwst(filters=filters, columns='*')

## Parse fields

Define "fields" of groups of exposures that overlap.

In [ ]:
tabs = overlaps.find_overlaps(res, use_parent=True, buffer_arcmin=2.5, close=False)

## Split into our own "associations" and make fancy figures

  1. Define associations by filter+pupile and NIRCam detector
  2. Figure accounts for multiple NIRCam detectors

In [ ]:
assoc_args = assoc_args={'max_pa': 2,
                         'max_sep': 0.5,
                         'max_time': 1.0,
                         'match_filter': True,
                         'match_instrument': True,
                         'match_program': True,
                         'hack_grism_pa': False,
                         'parse_for_grisms': False, 
                         'match_detector':True}

LS_ARGS = dict(pixscale=1,
               layers=['ls-dr9', 'sdss', 'unwise-neo7'],
               zorder=-1000,
               alpha=0.8,
               aspect='auto',
               verbose=True,
               grayscale=False,
               grayscale_params=[99, 1.5, -0.02])

for i, tab in enumerate(tabs[:]):
    _ = overlaps.split_associations(tab, assoc_args=assoc_args, force_split=True,
                                    with_ls_thumbnail=True, xsize=8, ls_args=LS_ARGS)
    
    _[1].savefig('{0}_footprint.png'.format(tab.meta['NAME']), dpi=150)
    tab.write('{0}_footprint.fits'.format(tab.meta['NAME']), overwrite=True)
    
    print(i, tab.meta['NAME'], len(np.unique(tab['assoc_idx'])))

In [ ]:
from grizli import utils
tab = tabs[-1]
assoc = utils.Unique(tab['assoc_idx'], verbose=False)
for a in assoc.values:
    print(f'\n assoc_idx = {a}\n================')
    for row in tab[assoc[a]]:
        print(f"{row['filename']:50s} {row['detector']:10} {row['filter-pupil']:14} {row['exptime']:6.1f}  {row['targname']}")